In [14]:
import datetime
from dateutil.relativedelta import relativedelta

import numpy as np
np.float = float    
np.int = int   #module 'numpy' has no attribute 'int'
np.object = object    #module 'numpy' has no attribute 'object'
np.bool = bool    
import pandas as pd 
import time
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options 
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
import re
from datetime import datetime,timedelta
from datetime import date
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import os.path
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pyautogui as pag
import shutil
from os.path import exists
from selenium.webdriver.support.ui import Select


import bets_lib as bets


In [41]:
#Below Functions need driver to point to tournament list: https://be.tournamentsoftware.com/find?DateFilterType=0&StartDate=2024-08-20&EndDate=2024-11-21&page=1
def load_all_tournaments(driver):
    try:
        loadMoreButton = driver.find_element(by=By.ID, value = 'elem_loadmore')
        while True:
            loadMoreButton.click()
            time.sleep(1)
    except:
        pass

def select_level(driver,level):
    xpath = ''
    if level.lower() == 'international':
        xpath = '/html/body/div[1]/section/form/div/div/div[2]/div[2]/div/div/ul/li[4]/fieldset/ul/li[1]/div/label'
    if level.lower() == 'nationals':
        xpath = '//*[@id="filter_tournaments"]/div[2]/div/div/ul/li[4]/fieldset/ul/li[2]/div/label'
    if level.lower() == 'gold':
        xpath = '//*[@id="filter_tournaments"]/div[2]/div/div/ul/li[4]/fieldset/ul/li[3]/div/label'
    if level.lower() == 'silver':
        xpath = '//*[@id="filter_tournaments"]/div[2]/div/div/ul/li[4]/fieldset/ul/li[4]/div/label'
    if level.lower() == 'bronze':
        xpath = '//*[@id="filter_tournaments"]/div[2]/div/div/ul/li[4]/fieldset/ul/li[5]/div/label'
    if level.lower() == 'tier 4':
        xpath = '//*[@id="filter_tournaments"]/div[2]/div/div/ul/li[4]/fieldset/ul/li[6]/div/label'

    if not xpath:
        print(f'Couldnt find "{level}"')
        return 0

    selectedBox = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, xpath))
    )
    selectedBox.click()

def select_age(driver, age):
    ageGroupXpath = '//*[@id="TournamentExtendedFilter_AgeGroupID"]'
    ageSelector = Select(driver.find_element(by=By.XPATH,value = ageGroupXpath))
    # Print out all the options in the select dropdown
    options = ageSelector.options
    ageOptions = []
    for option in options[1:]:
        if not 'Masters' in option.text:
            ageOptions.append(option.text) 

    if age in str(ageOptions):
        ageSelector.select_by_visible_text(age)




In [50]:
### Determine Tournaments to Check  ###

#get url to point to correct time period.
baselineUrl = 'https://be.tournamentsoftware.com/find?DateFilterType=0&StartDate=STARTDATE&EndDate=ENDDATE&page=1'
selectionDate = '2024-09-16'
checkFrom = selectionDate
checkUntilRaw = datetime.strptime(selectionDate,'%Y-%m-%d') - relativedelta(years=1)
checkUntil = checkUntilRaw.strftime('%Y-%m-%d')
tournamentsUrl = baselineUrl.replace('STARTDATE',checkUntil).replace('ENDDATE',checkFrom)

#load basic url
driver = webdriver.Firefox()
driver.get(tournamentsUrl)
bets.deal_with_cookies(driver)

 #Level
select_level(driver,'gold')
select_level(driver,'nationals')
 #Age group:
select_age(driver,'Senior') 

load_all_tournaments(driver)

time.sleep(1)
#Extract links:
linkRegex = re.compile('href=".*?"')
titleRegex = re.compile('title=".*?"')
source = driver.page_source
soup = BeautifulSoup(source,features='html.parser')
tournamentsHtml = soup.find_all('h4',class_= 'media__title')

for t in tournamentsHtml:
    tLink = re.findall(linkRegex,str(t))[0].replace('href=','').replace('"','')
    tLink = 'https://be.tournamentsoftware.com/tournament/' + tLink.split('=')[1]
    tTitle = re.findall(titleRegex,str(t))[0].replace('title=','').replace('"','')
    print(tLink,tTitle)



#https://be.tournamentsoftware.com/tournament/6257D6DC-E2C8-4174-BF9C-873AF182E875


/sport/tournament?id=77D53A72-E970-4A67-89EF-3B246B6A6316 The Middlesex Senior Gold Tournament 2023
/sport/tournament?id=065FF917-A6F8-41DB-B83F-1126141B5803 2024 Northumberland Senior Gold
/sport/tournament?id=1DF7E663-F865-4C7C-9F81-D48677060129 English National Badminton Championships 2024
/sport/tournament?id=1A3FCD02-6F1F-4CAE-BFFB-2800C6775225 Nottinghamshire Senior Gold 2024
/sport/tournament?id=0804AB30-445F-4F8E-B1CE-C6A076C82251 Dorset Senior Gold 2024
